In [1]:
%load_ext autoreload 
%autoreload 2 

In [4]:
import random
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # for subplots
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn.objects as so
import tensorflow as tf

from src.utilities.plot_helpers import generate_random_colors
from src.modeling.experiments.binned_trial_experiment_01 import split_data

from src.utilities.pandas_helpers import get_features
plt.rcdefaults()

2023-07-18 19:02:06.739159: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 19:02:06.819754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 19:02:09.782556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
DATA_PATH = '/projects/p31961/gaby_data/aggregated_data/data_pipeline_full_dataset/datasets/full_dataset.parquet.gzip'

In [6]:
data = pd.read_parquet(DATA_PATH)
data.head()

,time,mouse_id,sex_M,day,trial,learning_phase,event_cue,event_escape,event_avoid,latency,event_shock,sensor_D1,sensor_D2,sensor_DA,signal
0,-25.000000,12,0,5,0,0,1,0,0,0.0,0,0,1,0,-0.155359
1,-24.901531,12,0,5,0,0,1,0,0,0.0,0,0,1,0,-0.420553
2,-24.803064,12,0,5,0,0,1,0,0,0.0,0,0,1,0,-1.592294
3,-24.704596,12,0,5,0,0,1,0,0,0.0,0,0,1,0,-1.268734
4,-24.606127,12,0,5,0,0,1,0,0,0.0,0,0,1,0,-0.210176


In [37]:

def assign_cumulative_trials(df):

    # calculate total trials per mouse and store in dict to call later
    total_trials = {mouse: df.groupby(by = ['mouse_id', 'day', 'trial']).count().query("mouse_id == @mouse").shape[0] for mouse in df.mouse_id.unique()}
    # function to generate cumulative trial count for each mouse and store as independent dataframe: used to merge with original dataframe
    def get_trial_count(df, mouse, trials_dict):
        return(
            df
            .query("mouse_id == @mouse")
            .groupby(by = ['day', 'trial'], as_index = False).mean()
            .sort_values(by = ['day', 'trial'])
            .assign(trial_count = range(0, trials_dict[mouse]))
        )
    # concatenate all mouse dataframes into one and select only relevant columns to me merge with original dataframe        
    data_agg_trials = pd.concat([get_trial_count(df, mouse, total_trials) for mouse in df.mouse_id.unique()])[['mouse_id', 'day', 'trial', 'trial_count']]
    data_merged  = df.merge(data_agg_trials[["mouse_id", "trial", "trial_count", "day"]], how = "left")
    return data_merged
merged_data = assign_cumulative_trials(data)
merged_data


,time,mouse_id,sex_M,day,trial,learning_phase,event_cue,event_escape,event_avoid,latency,event_shock,sensor_D1,sensor_D2,sensor_DA,signal,trial_count
0,-25.000000,12,0,5,0,0,1,0,0,0.00,0,0,1,0,-0.155359,115
1,-24.901531,12,0,5,0,0,1,0,0,0.00,0,0,1,0,-0.420553,115
2,-24.803064,12,0,5,0,0,1,0,0,0.00,0,0,1,0,-1.592294,115
3,-24.704596,12,0,5,0,0,1,0,0,0.00,0,0,1,0,-1.268734,115
4,-24.606127,12,0,5,0,0,1,0,0,0.00,0,0,1,0,-0.210176,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858795,19.606127,1,0,4,1,0,0,1,0,7.54,0,1,0,0,0.299603,87
5858796,19.704596,1,0,4,1,0,0,1,0,7.54,0,1,0,0,-0.249408,87
5858797,19.803064,1,0,4,1,0,0,1,0,7.54,0,1,0,0,-0.486369,87
5858798,19.901531,1,0,4,1,0,0,1,0,7.54,0,1,0,0,-0.146374,87
